In [1]:
import cv2
import pandas as pd
import os

In [ ]:


# --- Configuration ---
VIDEO_NAME = 'Monica Greene unedited tennis match play.mp4' # <-- CHANGE THIS to your video's filename
ANNOTATIONS_DIR = 'annotations'
VIDEOS_DIR = 'raw_videos'
OUTPUT_DIR = 'sanity_check_clips' # A new folder for our output

# --- Setup Paths ---
annotation_path = os.path.join(ANNOTATIONS_DIR, f"{VIDEO_NAME}.csv")
video_path = os.path.join(VIDEOS_DIR, VIDEO_NAME)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Load the Annotation Data
print(f"Loading annotations from: {annotation_path}")
annotations = pd.read_csv(annotation_path)

# Take just the first annotated point for our test
first_point = annotations.iloc[0]
start_time_sec = first_point['startTime']
end_time_sec = first_point['endTime']

print(f"Found first point: Start at {start_time_sec:.2f}s, End at {end_time_sec:.2f}s")

# 2. Open the Video File
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video FPS: {fps:.2f}")

# 3. Calculate Frame Numbers
start_frame = int(start_time_sec * fps)
end_frame = int(end_time_sec * fps)
print(f"Extracting from frame {start_frame} to {end_frame}")

# 4. Extract and Write the Clip
output_clip_path = os.path.join(OUTPUT_DIR, f'clip_of_{VIDEO_NAME}')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_clip_path, fourcc, fps, (frame_width, frame_height))

cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
current_frame = start_frame

while current_frame <= end_frame:
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)
    current_frame += 1

# --- Cleanup ---
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ Sanity check clip successfully saved to: {output_clip_path}")